In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import kraft

In [ ]:
point_x_dimensions = (
    np.asarray(((0,), (1,), (2,),)),
    np.asarray(((0, 0), (1, 2), (2, 4),)),
    np.asarray(((0, 0, 0), (1, 2, 4), (2, 4, 8),)),
)

## get_probability

In [ ]:
for point_x_dimension in point_x_dimensions:

    print(kraft.get_probability(point_x_dimension))

## get_posterior_probability

In [ ]:
for point_x_dimension in point_x_dimensions:

    print(kraft.get_posterior_probability(point_x_dimension))

In [ ]:
for point_x_dimension in point_x_dimensions[1:]:

    print(
        kraft.get_posterior_probability(point_x_dimension, target_dimension_number=2.02)
    )

## plot_nomogram